# Arboles de clasificación y regresión simples vs bagging (ensambles)

En este lab vamos a comparar el rendimiento de un caso de árbol de **clasificación** y otro de **regresión** con el rendimiento de cada versión implementando **bagging**.

El método de bagging busca generar un conjunto de modelos que en promedio predigan mejor, utilizando técnicas de resampling (bootstrapping) sobre el set de muestras de entrenamiento.

En el caso de la regresión, tendremos un promedio de los outputs de todos los modelos. En el caso de la clasificación, el voto de la mayoría determinará el resultado binario en este caso. 

La ventaja es la reducción de la varianza o overfitting y corrigen así este problema que tienen los árboles en general logrando una combinación exitosa.

Usaremos dos datasets:

Clasificación, utilizaremos el dataset cancer de mama de sklearn.

Regresión, utilizaremos el dataset diabetes también de sklearn.

### Modelo de clasificación

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [19]:
#Levantamos el dataset breast cancer de sklearn
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
y = data['target']
X = data['data']

In [20]:
#Seteamos parámetros para CV: Folds y porcentaje train/test

cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

In [21]:
#instanciamos y entrenamos un arbol de clasificación y un arbol de clasificación con bagging

dt = DecisionTreeClassifier(class_weight='balanced', random_state=1)
dt.fit(X_train, y_train)

bdt = BaggingClassifier(DecisionTreeClassifier())
bdt.fit(X_train, y_train)


BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
    

In [27]:
#probamos la performance de los árboles en el conjunto test con accuracy y matriz de confusión

dt_y_pred = dt.predict(X_test)
print('Accuracy dt=', accuracy_score(y_test, dt_y_pred))
confusiondt = confusion_matrix(y_test, dt_y_pred)
print(confusiondt)

print ()

bdt_y_pred = bdt.predict(X_test)
print('Accuracy bdt=', accuracy_score(y_test, bdt_y_pred))
confusionbdt = confusion_matrix(y_test, bdt_y_pred)
print(confusionbdt)


Accuracy dt= 0.9415204678362573
[[ 57   4]
 [  6 104]]

Accuracy bdt= 0.9766081871345029
[[ 60   1]
 [  3 107]]


##### Vemos una mejora clara en el resultado del clasificador con bagging

Ahora podemos probar un tipo de bagging específico llamado Random Forest. Toma subconjuntos aleatorios de features para cada caso. Para buscar hiperparámetros, vamos a utilizar Gridsearch y tratar de mejorar el resultado anterior!

In [8]:
#instanciamos el clasificador y definimos el rango de iteración para el Gridsearch

rf = RandomForestClassifier(class_weight='balanced', random_state=1)

from sklearn.model_selection import GridSearchCV
param_trees = {'n_estimators': [50, 100, 200], 
               'max_features': [1, 5, 8, 10, 21], 
               'max_depth': [5, 20, 50, 70, 100], 
               'min_samples_leaf':[1, 5, 8, 10, 50]}
grid_search_rf = GridSearchCV(rf, param_grid=param_trees, cv=cv , verbose=1, n_jobs=-1)


In [9]:
#entrenamos el modelo

grid_search_rf.fit(X_train, y_train)

Fitting 5 folds for each of 375 candidates, totalling 1875 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1875 out of 1875 | elapsed:  5.8min finished
C:\Users\Joaquin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=41, shuffle=True),
             error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=1,
                   

In [10]:
#Probamos el rendimiento en test

grid_search_rf_y_pred = grid_search_rf.predict(X_test)
print('Accuracy grid_search_rf =', accuracy_score(y_test, grid_search_rf_y_pred))
confusion_grid_search_rf = confusion_matrix(y_test, grid_search_rf_y_pred)
print(confusion_grid_search_rf)


Accuracy grid_search_rf = 0.9941520467836257
[[ 61   0]
 [  1 109]]


#### Logramos una clasificación casi perfecta! 

### Modelo de regresión

In [11]:
#Instalamos las librerías necesarias e importamos el set de datos Diabetes

from sklearn.datasets import load_diabetes
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

data = load_diabetes()
X = data['data']
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

In [12]:
#Instanciamos el arbol de decisión para regresión y su versión con bagging
#Fiteamos ambos modelos

dtr = DecisionTreeRegressor(max_depth=2)
dtr.fit(X_train, y_train)

bdtr = BaggingRegressor(dtr)
bdtr.fit(X_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                      max_depth=2,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort=False,
                                                      random_state=None,
                                                      splitter='best'),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=1

In [13]:
#Evaluamos el rendimiento en el conjunto TEST de los dos modelos

from sklearn.metrics import r2_score

dtr_y_pred = dtr.predict(X_test)
print('Accuracy dtr=', r2_score(y_test, dtr_y_pred))

bdtr_y_pred = bdtr.predict(X_test)
print('Accuracy bdtr=', r2_score(y_test, bdtr_y_pred))

Accuracy dtr= 0.2604742200567519
Accuracy bdtr= 0.3667665022817359


##### Utilizando la métrica R2 para comparar, vemos una mejora visible de la implementación con bagging!